In [14]:
import numpy as np
import pandas as pd
eps = np.finfo(float).eps
from numpy import log2 as log
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

In [15]:
df = pd.read_csv('F:\\VIT\\WINTER 2018-19\\DATA MINING\\Diabetes_Prediction-master\\Dataset\\diabetes.csv')

In [16]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
5,5,116,74,0,0,25.6,0.201,30,0
6,3,78,50,32,88,31.0,0.248,26,1
7,10,115,0,0,0,35.3,0.134,29,0
8,2,197,70,45,543,30.5,0.158,53,1
9,8,125,96,0,0,0.0,0.232,54,1


In [25]:
def find_entropy(df):
    Class = df.keys()[-1]   
    entropy = 0
    values = df[Class].unique()
    for value in values:
        fraction = df[Class].value_counts()[value]/len(df[Class])
        entropy += -fraction*np.log2(fraction)
    return entropy

In [26]:
def find_entropy_attribute(df,attribute):
  Class = df.keys()[-1]   
  target_variables = df[Class].unique()
  variables = df[attribute].unique()    
  entropy2 = 0
  for variable in variables:
      entropy = 0
      for target_variable in target_variables:
          num = len(df[attribute][df[attribute]==variable][df[Class] ==target_variable])
          den = len(df[attribute][df[attribute]==variable])
          fraction = num/(den+eps)
          entropy += -fraction*log(fraction+eps)
      fraction2 = den/len(df)
      entropy2 += -fraction2*entropy
  return abs(entropy2)



In [27]:
def find_winner(df):
    Entropy_att = []
    IG = []
    for key in df.keys()[:-1]:
        IG.append(find_entropy(df)-find_entropy_attribute(df,key))
    return df.keys()[:-1][np.argmax(IG)]

In [28]:
def get_subtable(df, node,value):
  return df[df[node] == value].reset_index(drop=True)

In [29]:
def buildTree(df,tree=None): 
    Class = df.keys()[-1]   

    node = find_winner(df)
    
    attValue = np.unique(df[node])
     
    if tree is None:                    
        tree={}
        tree[node] = {}
    

    for value in attValue:
        
        subtable = get_subtable(df,node,value)
        clValue,counts = np.unique(subtable['Outcome'],return_counts=True)                        
        
        if len(counts)==1:
            tree[node][value] = clValue[0]                                                    
        else:        
            tree[node][value] = buildTree(subtable) 
                   
    return tree
  

In [30]:
tree = buildTree(df)

In [31]:
import pprint
pprint.pprint(tree)

{'DiabetesPedigreeFunction': {0.078: 0,
                              0.084: 0,
                              0.085: 0,
                              0.08800000000000001: {'Pregnancies': {1: 1,
                                                                    2: 0}},
                              0.08900000000000001: 0,
                              0.092: 0,
                              0.096: 0,
                              0.1: 0,
                              0.10099999999999999: 0,
                              0.102: 0,
                              0.107: 0,
                              0.10800000000000001: 0,
                              0.115: 0,
                              0.11800000000000001: 0,
                              0.121: {'Pregnancies': {3: 0, 6: 1}},
                              0.122: 0,
                              0.12300000000000001: 0,
                              0.126: 0,
                              0.127: 1,
                              0.12

In [20]:
find_winner(df)

'DiabetesPedigreeFunction'

In [22]:
X = df.iloc[:, :-1].values
y = df.iloc[:, 8].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)


from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


treeclf = DecisionTreeClassifier(random_state=42)
parameters = {'max_depth': [6, 7, 8, 9],
              'min_samples_split': [2, 3, 4, 5],
              'max_features': [1, 2, 3, 4]
}

In [24]:
X_train,X_test,y_train,y_test = train_test_split(X, y, random_state=42)
tree.fit(X_train, y_train)
print("Accuracy on training set: {:.3f}".format(tree.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(tree.score(X_test, y_test)))


Accuracy on training set: 0.852
Accuracy on test set: 0.729
